In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 Vertex AI 群集管理中使用 Ray

<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/ray_cluster_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> <br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fray_on_vertex_ai%2Fray_cluster_management.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"> <br> 在 Colab Enterprise 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ray_on_vertex_ai/ray_cluster_management.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> <br>
      在 Vertex AI Workbench 中打开
    </a>
  </td>
<td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/ray_cluster_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> <br>
      在 GitHub 中查看
    </a>
  </td>
</table>

## 概览

本教程演示了如何使用Ray on Vertex AI SDK进行集群管理。

了解有关[Ray on Vertex AI概览](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview)的更多信息。

### 目标

在本教程中，您将学习如何创建一个集群，列出现有的集群，获取一个集群，手动调整（扩展规模）一个集群，并删除一个集群。

本教程使用以下 Vertex AI 服务和资源：

- [Vertex AI 上的 Ray](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview)

执行的步骤包括：

- 创建一个集群。
- 列出现有集群。
- 获取一个集群。
- 手动扩展集群，然后缩小集群规模。
- 删除现有集群。

成本

此教程使用Google Cloud的付费组件：

* Vertex AI

了解[Vertex AI定价上的Ray](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview#pricing)，
并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

## 开始使用

### 安装 Vertex AI SDK 和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[ray]

重新启动运行时（仅限Colab）

为了使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️核心即将重新启动。 请等待完成后再继续下一步。⚠️</b>
</div>

### 在Google Colab中验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息并初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的谷歌云项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

### 设置网络信息

[建立 VPC 对等网络](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) 和私有服务连接以访问 Vertex AI。

In [ ]:
VPC_NETWORK = "[your-network-name]"  # @param {type:"string"}
VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

###导入库

In [ ]:
import vertex_ray

创建一个集群

请注意在同一个VPC网络中，IP范围限制了您可以创建的集群和节点数量。

In [ ]:
head_node_type = vertex_ray.Resources(
    machine_type="n1-standard-16",
    node_count=1,
)

worker_node_types = [
    vertex_ray.Resources(
        machine_type="n1-standard-8",
        node_count=2,  # Can be > 1
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
    )
]

cluster_resource_name = vertex_ray.create_ray_cluster(
    head_node_type=head_node_type,
    network=VPC_NETWORK_FULL,
    worker_node_types=worker_node_types,
)

## 列出现有的集群

In [ ]:
clusters = vertex_ray.list_ray_clusters()
clusters

更新现有的集群（手动扩展）

请注意，您可以扩展的最大工作节点数取决于初始节点计数（更多详细信息请查看这些[公式](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/set-up)），并受限于同一VPC网络中的IP范围。

获取您想要扩展的集群。

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

将工人数量从2个节点减少到1个节点。

In [ ]:
new_worker_node_types = []
for worker_node_type in cluster.worker_node_types:
    worker_node_type.node_count = 1
    new_worker_node_types.append(worker_node_type)

cluster_resource_name = vertex_ray.update_ray_cluster(
    cluster_resource_name=cluster_resource_name,
    worker_node_types=new_worker_node_types,
)

验证集群是否成功缩减。

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

扩展到2个工作节点。

In [ ]:
new_worker_node_types = []
for worker_node_type in cluster.worker_node_types:
    worker_node_type.node_count = 2
    new_worker_node_types.append(worker_node_type)

cluster_resource_name = vertex_ray.update_ray_cluster(
    cluster_resource_name=cluster_resource_name,
    worker_node_types=new_worker_node_types,
)

验证集群已成功扩容。

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

清理工作

要清理项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的集群。

In [ ]:
# Delete the cluster
vertex_ray.delete_ray_cluster(cluster.cluster_resource_name)